In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

response = client.invoke([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンといいます。"}
])
print(response)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1",
    streaming=True
)

response = client.invoke([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンといいます。"},

])
print(response)


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1",
    response_mime_type="application/json"
)

response = client.invoke([
    {"role": "system", "content": "人物一覧を次のJSON形式で出力してください。[{\"name\": \"\", \"age\": \"\", \"city\": \"\"}]"},
    {"role": "user", "content": "以下の人物について教えてください。佐藤太郎50歳、東京都出身です。普段は会社に勤めてます。鈴木花子30歳、大阪府出身です。フリーランスで働いています。"},
])

print(response.content)

In [ ]:
# ====================================
# Function Calling 入門
# ====================================
# AIが「どの関数を呼ぶべきか」を判断し、引数を生成する機能

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

# 1. ツール（関数）を定義
@tool
def get_weather(city: str) -> str:
    """指定した都市の天気を取得する"""
    # 実際はAPIを呼ぶが、ここではダミーデータ
    weather_data = {
        "東京": "晴れ 25℃",
        "大阪": "曇り 22℃",
        "名古屋": "雨 18℃"
    }
    return weather_data.get(city, f"{city}の天気情報はありません")

@tool
def calculate(expression: str) -> str:
    """数式を計算する"""
    try:
        result = eval(expression)
        return f"計算結果: {result}"
    except:
        return "計算できませんでした"

# 2. LLMにツールをバインド
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

llm_with_tools = llm.bind_tools([get_weather, calculate])

# 3. 質問してみる
response = llm_with_tools.invoke("大分県の天気は？")
#今まではresponse = llm.invoke{[
#    {"role": "system", "content": "人物一覧を次のJSON形式で出力してください。[{\"name\": \"\", \"age\": \"\", \"city\": \"\"}]"},
#    {"role": "user", "content": "以下の人物について教えてください。佐藤太郎50歳、東京都出身です。普段は会社に勤めてます。鈴木花子30歳、大阪府出身です。フリーランスで働いています。"},
#]}


print("=== AIの判断 ===")
print(f"呼び出すツール: {response.tool_calls}")
print()

# 4. ツールを実際に実行
if response.tool_calls:
    tool_call = response.tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    print(f"=== ツール実行 ===")
    print(f"ツール名: {tool_name}")
    print(f"引数: {tool_args}")

    # ツールを実行
    if tool_name == "get_weather":
        result = get_weather.invoke(tool_args)
    elif tool_name == "calculate":
        result = calculate.invoke(tool_args)

    print(f"結果: {result}")

In [ ]:
import json

def get_current_weather(location,unint="fahrenheit"):
    if "tokyo" in location.lower():
        return {
            "location": "Tokyo",
            "temperature": "22",
            "unit": unint,
            "forecast": "sunny"
        }
    elif "osaka" in location.lower():
        return {
            "location": "Osaka",
            "temperature": "25",
            "unit": unint,
            "forecast": "cloudy"
        }
    else:
        return {
            "location": location,
            "temperature": "unknown",
            "unit": unint,
            "forecast": "unknown"
        }

In [ ]:
# ====================================
# Function Calling × SQL
# ====================================
# 自然言語からSQLを生成・実行する

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import sqlite3

# 1. インメモリDBを作成してサンプルデータを投入
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        city TEXT,
        department TEXT
    )
""")

# サンプルデータ
sample_users = [
    ("山田太郎", 30, "東京", "営業部"),
    ("鈴木花子", 25, "大阪", "開発部"),
    ("佐藤次郎", 28, "名古屋", "営業部"),
    ("田中美咲", 35, "東京", "人事部"),
    ("高橋健一", 42, "福岡", "開発部"),
    ("伊藤さくら", 23, "東京", "開発部"),
]

cursor.executemany("INSERT INTO users (name, age, city, department) VALUES (?, ?, ?, ?)", sample_users)
conn.commit()

print("=== データベース作成完了 ===")
print("テーブル: users (id, name, age, city, department)")
print(f"データ件数: {len(sample_users)}件")
print()

# 全データ表示
cursor.execute("SELECT * FROM users")
for row in cursor.fetchall():
    print(row)

In [ ]:
# 2. SQLを実行するツールを定義
@tool
def execute_sql(query: str) -> str:
    """
    SQLクエリを実行してデータベースから情報を取得する。

    テーブル: users
    カラム: id, name(名前), age(年齢), city(都市), department(部署)

    例:
    - SELECT * FROM users WHERE city = '東京'
    - SELECT * FROM users WHERE age >= 30
    - SELECT department, COUNT(*) FROM users GROUP BY department
    """
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        if results:
            columns = [desc[0] for desc in cursor.description]
            output = f"カラム: {columns}\n"
            for row in results:
                output += f"{row}\n"
            return output
        return "該当するデータがありませんでした"
    except Exception as e:
        return f"エラー: {e}"

# 3. LLMにツールをバインド
llm_sql = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

llm_with_sql = llm_sql.bind_tools([execute_sql])

print("=== SQLツール設定完了 ===")

In [ ]:
# 4. 自然言語で質問してみる
def ask_database(question: str):
    """自然言語でDBに質問する"""
    print(f"質問: {question}")
    print("-" * 40)

    response = llm_with_sql.invoke(question)

    if response.tool_calls:
        tool_call = response.tool_calls[0]
        sql_query = tool_call["args"]["query"]

        print(f"生成されたSQL: {sql_query}")
        print("-" * 40)

        result = execute_sql.invoke({"query": sql_query})
        print(f"結果:\n{result}")
    else:
        print(f"AIの回答: {response.content}")

    print("=" * 50)
    print()

# いくつかの質問を試す
ask_database("東京に住んでいる人を全員教えて")

In [ ]:
# 他の質問も試してみよう
ask_database("30歳以上の社員は誰？")
ask_database("部署ごとの人数を教えて")
ask_database("開発部で一番若い人は？")

In [75]:
prompt ='''\
以下の料理のレシピを教えてください。

料理名: """
{dish}
"""
'''

def get_recipe(dish_name: str) -> str:
    response = llm.invoke([
        {"role": "user", "content": prompt.format(dish=dish_name)}
    ])
    return response.content

get_recipe("カレーライス")
#prompt.format(dish="田中")

'はい、カレーライスのレシピです。基本のレシピからアレンジまでご紹介します。\n\n**基本のカレーライス**\n\n**材料 (4～5人分)**\n\n*   **肉:**\n    *   牛肉、豚肉、鶏肉など (カレー用)：300～400g\n*   **野菜:**\n    *   玉ねぎ：2個 (薄切り)\n    *   じゃがいも：2～3個 (乱切り)\n    *   にんじん：1本 (乱切り)\n*   **その他:**\n    *   カレールー：1箱 (8～12皿分)\n    *   サラダ油：大さじ2\n    *   水：ルーの箱の表示に従う\n    *   ご飯：適量\n\n**作り方**\n\n1.  **下準備:**\n    *   肉は大きければ食べやすい大きさに切る。\n    *   野菜はそれぞれ皮をむき、乱切りにする。玉ねぎは薄切りにする。\n2.  **炒め:**\n    *   鍋にサラダ油を熱し、玉ねぎを炒める。焦げ付かないように弱火～中火で、飴色になるまでじっくり炒める (約10～15分)。\n    *   肉を加えて炒め、表面の色が変わったら、じゃがいも、にんじんを加えてさらに炒める。\n3.  **煮込み:**\n    *   野菜全体に油が回ったら、ルーの箱に表示されている分量の水を加えて沸騰させる。\n    *   アクが出てきたらすくい取り、弱火～中火で野菜が柔らかくなるまで煮込む (約15～20分)。\n4.  **ルーを溶かす:**\n    *   いったん火を止め、カレールーを割り入れて溶かす。\n    *   再び弱火でとろみがつくまで煮込む (約5～10分)。焦げ付かないように時々混ぜる。\n5.  **盛り付け:**\n    *   ご飯を皿に盛り、カレーをかければ完成。\n\n**美味しく作るためのポイント**\n\n*   **玉ねぎをしっかり炒める:** 玉ねぎを飴色になるまで炒めることで、カレーに甘みとコクが出ます。焦げ付かないように注意しながら、じっくり炒めましょう。\n*   **隠し味を加える:** チョコレート、コーヒー、ソース、ヨーグルト、ケチャップなどを少量加えると、味に深みが増します。\n*   **煮込み時間を調整する:** 野菜の硬さはお好みで調整

In [76]:
# ====================================
# LangSmith トレース例（langchain-google-vertexai使用）
# ====================================

from dotenv import load_dotenv
load_dotenv()  # LangSmithの設定を読み込む

from langchain_google_genai import ChatGoogleGenerativeAI

# Vertex AI経由でGeminiを使用
llm_vertex = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 基本的な使い方
response = llm_vertex.invoke("こんにちは！今日の気分はどう？")
print("=== 基本的な呼び出し ===")
print(response.content)
print()

# メッセージ形式
response2 = llm_vertex.invoke([
    {"role": "system", "content": "あなたは関西弁で話すアシスタントです。"},
    {"role": "user", "content": "自己紹介してください"}
])
print("=== システムプロンプト付き ===")
print(response2.content)

=== 基本的な呼び出し ===
こんにちは！今日の気分はとても良いです！ 😊 あなたはいかがですか？

=== システムプロンプト付き ===
へい、毎度！わいは、皆さんの頼りになる関西弁アシスタントや！困ったこと、知りたいこと、何でもかんでも気軽に聞いてな！おもろい話も大好きやで！


In [77]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["""以下の料理のレシピを考えてください。料理名 {dish}"""
])

prompt_value = prompt.invoke({"dish": "カレー"})


In [78]:
# ChatPromptValue の中身を確認する方法
print("=== 文字列として表示 ===")
print(prompt_value.to_string())

print("\n=== メッセージとして表示 ===")
print(prompt_value.to_messages())

=== 文字列として表示 ===
Human: 以下の料理のレシピを考えてください。料理名 カレー

=== メッセージとして表示 ===
[HumanMessage(content='以下の料理のレシピを考えてください。料理名 カレー', additional_kwargs={}, response_metadata={})]


In [79]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    ("human", "以下の料理のレシピを考えてください。料理名 {dish}")
])

prompt_value = prompt.invoke({"dish": "オムライス"})


In [80]:
print("=== 文字列として表示 ===")
print(prompt_value.to_string())

=== 文字列として表示 ===
System: あなたは料理の専門家です。
Human: 以下の料理のレシピを考えてください。料理名 オムライス


In [81]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_core.tools import tool

prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    MessagesPlaceholder(variable_name="history", optional=True),
    ("human", "以下の料理のレシピを考えてください。料理名 {dish}")
])

prompt_value = prompt.invoke({"dish": "オムライス"})
print("=== 文字列として表示 ===")
print(prompt_value.to_string())

=== 文字列として表示 ===
System: あなたは料理の専門家です。
Human: 以下の料理のレシピを考えてください。料理名 オムライス


In [82]:
prompt_value = prompt.invoke({
    "dish": "カレー",
    "history": [
        ("human", "カレーのレシピを教えて"),
        ("ai", "カレーのレシピは玉ねぎを炒めて..."),
        ("human", "辛くするには？"),
        ("ai", "スパイスを増やすと辛くなります"),
        ("human", "激辛が好きなんだよ"),
    ]
})

In [ ]:
print("=== 文字列として表示 ===")
print(llm.invoke(prompt_value).content)

=== 文字列として表示 ===


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# 準備
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です"),
    ("human", "{dish}のレシピを教えて")
])

# 実行（パイプでつなぐ）
chain = prompt | llm
result = chain.invoke({"dish": "カレー"})
print(result.content)

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser


class Recipe(BaseModel):
  ingredients: list[str] = Field(description="料理の材料のリスト")
  steps: list[str] = Field(description="料理の手順のリスト")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
print("=== 文字列として表示 ===")
print(format_instructions)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 型定義
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="料理の材料のリスト")
    steps: list[str] = Field(description="料理の手順のリスト")

# パーサー
output_parser = PydanticOutputParser(pydantic_object=Recipe)

# LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# プロンプト
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    ("human", "以下の料理のレシピを考えてください。料理名 {dish}\n\n{format_instructions}")
])

# チェーン（output_parserを追加）
chain = prompt | llm | output_parser

# 実行
result = chain.invoke({
    "dish": "オムライス",
    "format_instructions": output_parser.get_format_instructions()
})

print(result)
# → Recipe(ingredients=[...], steps=[...])


In [ ]:
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="料理の材料のリスト")
    steps: list[str] = Field(description="料理の手順のリスト")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# これだけでOK！
structured_llm = llm.with_structured_output(Recipe)
result = structured_llm.invoke("オムライスのレシピを教えて")

print(result.ingredients)
print(result.steps)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 1つ目: 料理名を考える
prompt1 = ChatPromptTemplate.from_template("{ingredient}を使った料理名を1つだけ答えて")

# 2つ目: その料理のレシピを考える
prompt2 = ChatPromptTemplate.from_template("{dish}のレシピを教えて")

# チェーンをつなげる
chain = (
    prompt1
    | llm
    | StrOutputParser()  # ← ここで文字列に変換
    | (lambda dish: {"dish": dish})  # ← 次のプロンプトに渡す
    | prompt2
    | llm
    | StrOutputParser()
)

result = chain.invoke({"ingredient": "卵"})
print(result)
# 1. "卵"を使った料理名 → "オムライス"
# 2. "オムライス"のレシピ → "材料は..."


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 型定義
class Recipe(BaseModel):
    date: list[str] = Field(description="日付")
    menue: list[str] = Field(description="料理名")
    ingredients: list[str] = Field(description="料理の材料のリスト")
    steps: list[str] = Field(description="料理の手順のリスト")

# 1つ目: 料理名を考える
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。料理名だけを1つ答えてください。"),
    ("human", "{ingredient}を使った料理は？")
])

# 2つ目: レシピを構造化して取得
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "あなたは料理の専門家です。"),
    ("human", "{dish}のレシピを教えて")
])

structured_llm = llm.with_structured_output(Recipe)

# チェーン
chain = (
    prompt1
    | llm
    | StrOutputParser()
    | (lambda dish: {"dish": dish})  # ← 辞書に変換
    | prompt2
    | structured_llm
)

result = chain.invoke({"ingredient": "卵"})
print(result.date)
print(result.menue)
print(result.ingredients)
print(result.steps)


In [ ]:
# ====================================
# RAG（Retrieval-Augmented Generation）入門
# ====================================
# 自分のデータを検索して、LLMに回答させる仕組み

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM準備
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# ====================================
# 簡易版: ダミーのRetrieverを作成
# ====================================
# 本番では Vertex AI Search や ChromaDB を使う
# ここでは仕組みを理解するために、簡単な検索関数を作成

# サンプルデータ（本番ではVector Storeに保存）
documents = {
    "りんご": "りんごは青森県が生産量日本一です。糖度が高く、ふじ、つがる、王林などの品種があります。旬は秋から冬です。",
    "みかん": "みかんは和歌山県や愛媛県が有名な産地です。ビタミンCが豊富で、冬の定番果物です。",
    "バナナ": "バナナはフィリピンやエクアドルからの輸入が多いです。栄養価が高く、手軽に食べられる果物です。",
    "ぶどう": "ぶどうは山梨県が有名です。巨峰やシャインマスカットなどの品種があります。ワインの原料にもなります。",
}

def simple_retriever(query: str) -> str:
    """簡易検索: キーワードが含まれるドキュメントを返す"""
    results = []
    for keyword, content in documents.items():
        if keyword in query or any(word in query for word in ["果物", "フルーツ", "産地"]):
            results.append(f"【{keyword}について】\n{content}")

    if results:
        return "\n\n".join(results)
    return "該当する情報が見つかりませんでした。"

# ====================================
# RAGチェーンの構築
# ====================================

# プロンプト
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは果物の専門家です。以下の検索結果を元に質問に答えてください。検索結果にない情報は「わかりません」と答えてください。"),
    ("human", "検索結果:\n{context}\n\n質問: {question}")
])

# チェーン
# {"context": ..., "question": ...} の形でpromptに渡す
chain = (
    {
        "context": lambda x: simple_retriever(x),  # 検索実行
        "question": lambda x: x                     # 質問はそのまま（これが主流）
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ====================================
# 実行例
# ====================================
print("=== RAG実行例 ===\n")

# 質問1: データに含まれる情報
question1 = "りんごとみかんの産地はどこ？"
print(f"質問: {question1}")
print(f"回答: {chain.invoke(question1)}\n")
print("-" * 50)

# 質問2: データに含まれない情報
question2 = "いちごの産地はどこ？"
print(f"質問: {question2}")
print(f"回答: {chain.invoke(question2)}\n")

In [ ]:
from  langchain_google_genai import ChatGoogleGenerativeAI
client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import chain

In [ ]:
# ====================================
# Runnable: RunnableLambda と @chain
# ====================================
# パイプラインで関数を使う方法

from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import chain

# ------------------------------------
# 方法1: lambda（最も一般的）
# ------------------------------------
# パイプライン内で直接書ける

chain1 = (
    (lambda x: x.upper())       # 大文字に変換
    | (lambda x: f"結果: {x}")  # 文字列を整形
)
print(chain1.invoke("hello"))  # → "結果: HELLO"

# ------------------------------------
# 方法2: RunnableLambda（明示的に書く場合）
# ------------------------------------
# lambda と同じだが、より明示的

chain2 = (
    RunnableLambda(lambda x: x.upper())
    | RunnableLambda(lambda x: f"結果: {x}")
)
print(chain2.invoke("hello"))  # → "結果: HELLO"

# ------------------------------------
# 方法3: @chain デコレータ（複雑な処理の場合）
# ------------------------------------
# 関数が複雑な場合に使う（あまり使わない）

@chain
def process_data(data):
    """複数ステップの処理を1つの関数にまとめる"""
    step1 = data.upper()
    step2 = f"結果: {step1}"
    return step2

print(process_data.invoke("hello"))  # → "結果: HELLO"

# ------------------------------------
# 実践: パイプラインでの使用例
# ------------------------------------
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

prompt = ChatPromptTemplate.from_template("{dish}のレシピを教えて")

# lambda を使って次のステップに辞書を渡す
chain_with_lambda = (
    (lambda x: {"dish": x})  # 文字列 → 辞書に変換
    | prompt
    | llm
    | StrOutputParser()
)

# 文字列で直接呼べる
result = chain_with_lambda.invoke("オムライス")
print(result[:100] + "...")  # 最初の100文字だけ表示

In [ ]:
# ====================================
# Python基礎: クラスとインスタンス
# ====================================
# LangChainを理解する上で必要なPythonの基礎知識

# ------------------------------------
# クラス = 設計図
# インスタンス = 設計図から作った実体
# ------------------------------------

class Dog:
    # __init__ = 初期化（インスタンス作成時に自動実行）
    # 定義は書くが、呼び出しは書かない
    def __init__(self, name):
        self.name = name  # self = このインスタンス自体

    # 普通のメソッド（自分で呼ぶ）
    def bark(self):
        return f"{self.name}がワン！と鳴いた"

# インスタンス作成
dog1 = Dog("ポチ")   # → __init__(self=新しい箱, name="ポチ") が自動実行
dog2 = Dog("タロウ")  # → __init__(self=新しい箱, name="タロウ") が自動実行

# メソッド呼び出し
print(dog1.bark())  # → "ポチがワン！と鳴いた"（self = dog1）
print(dog2.bark())  # → "タロウがワン！と鳴いた"（self = dog2）

# ------------------------------------
# selfの理解
# ------------------------------------
# dog1.bark() が呼ばれると
#   → bark(self=dog1) として実行される
#   → self.name は dog1.name = "ポチ"

# ------------------------------------
# __init__ がないクラス（データを持たない場合）
# ------------------------------------
class Calculator:
    def add(self, a, b):
        return a + b

calc = Calculator()  # 何も渡さなくてOK
print(calc.add(1, 2))  # → 3

# ------------------------------------
# @（デコレータ）
# ------------------------------------
# 関数を「ラップ」して追加機能を付ける構文
# LangChainでは @tool がよく使われる

from langchain_core.tools import tool

@tool
def get_weather(city: str) -> str:
    """指定した都市の天気を取得する"""
    return f"{city}の天気は晴れです"

# @tool を付けると .invoke() で呼べるようになる
print(get_weather.invoke({"city": "東京"}))  # → "東京の天気は晴れです"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
# 並列実行のチェーン例


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 材料を聞くプロンプト
prompt_ingredients = ChatPromptTemplate.from_template(
    "{dish}の材料を3つだけ箇条書きで"
)

# カロリーを聞くプロンプト
prompt_calories = ChatPromptTemplate.from_template(
    "{dish}のカロリーを数値だけで答えて"
)

# TODO: 並列実行するチェーンを作成
chain = (
    (lambda x: {"dish": x})
    | RunnableParallel(
        ingredients=prompt_ingredients | llm | StrOutputParser(),
        calories=prompt_calories | llm | StrOutputParser(),
    )
)
result = chain.invoke("カレー")

print(result["ingredients"]),
print("==========")
print(result["calories"])



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# 条件分岐のチェーン例
def is_food_question(x):
    dish = x["dish"]  # ← ここを追加するだけ
    if "カレー" in dish or "作り方" in dish or "レシピ" in dish:
        return True
    return False

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 材料を聞くプロンプト
prompt_ingredients = ChatPromptTemplate.from_template(
    "料理に関する情報：{dish}"
)


# TODO: 条件分岐するチェーンを作成
chain = (
    (lambda x: {"dish": x})
    | RunnableBranch(
        (is_food_question, prompt_ingredients | llm | StrOutputParser()),
        lambda x: "料理に関する質問ではありません。"
    )
)
result = chain.invoke("カレーの作り方について教えて")

print(result)


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

# 材料を聞くプロンプト
prompt_dish = ChatPromptTemplate.from_template(
    "料理に関する情報：{dish}"
)


# TODO: 条件分岐するチェーンを作成
chain = (
    (lambda x: {"dish": x})
    | RunnablePassthrough.assign(new_key=prompt_dish | llm | StrOutputParser())
)
result = chain.invoke("カレーの作り方について教えて")

print(result),
print("==========")


chain2 = (
    (lambda x: {"dish": x})| prompt_dish | llm | StrOutputParser()
)
result2 = chain2.invoke("カレーの作り方について教えて")
print(result2),


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_community.document_loaders import GitLoader
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    project="dify-chatbot-469403",
    location="us-central1"
)

def file_filter(file_path: str) -> bool:
    """特定のファイルだけを処理するフィルタ関数"""
    return file_path.endswith(".md")  # 例: .mdファイルだけ処理

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter
)

documents = loader.load()[:10]
print(f"読み込んだドキュメント数: {len(documents)}")

from langchain_chroma import Chroma
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embedding
embeddings = VertexAIEmbeddings(
    model_name="text-embedding-004",
    project="dify-chatbot-469403",
    location="us-central1"
)

# Vector Store作成
db = Chroma.from_documents(
    documents,
    embedding=embeddings,
    collection_name="langchain_docs"
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages([
    ("system", "以下のドキュメントを参考に質問に答えてください。"),
    ("human", "ドキュメント:\n{context}\n\n質問: {question}")
])

retriever = db.as_retriever()

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("LangChainとは何ですか？")

In [ ]:
chain = (
    (lambda x: {"question": x})
    | RunnablePassthrough.assign(context=retriever)
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")

query_generation_prompt = ChatPromptTemplate.from_messages(["""\
質問に対してベクターデータベースで検索するための適切な検索クエリを3つ生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して多様な観点からアプローチするクエリを作成します。

質問: {question}
"""])

query_generation_chain = (
    query_generation_prompt
    | llm.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)


In [ ]:

multi_query_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "queries": query_generation_chain,
    }
    | (lambda x: {
        "question": x["question"],
        "context": sum([retriever.invoke(q) for q in x["queries"]], [])
    })
    | prompt
    | llm
    | StrOutputParser()
)
